### Libraries

In [ ]:
import os
import wfdb
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Conv2D, MaxPooling2D, Reshape, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import time




### NNs & DNNs

In [ ]:
# Define the path to the extracted dataset
data_path = 'M:\Dissertation\mit-bih-arrhythmia-database-1.0.0-20240722T094228Z-001\mit-bih-arrhythmia-database-1.0.0' 

# Function to load a record and preprocess
def load_and_preprocess(record):
    signal, fields = wfdb.rdsamp(os.path.join(data_path, record))
    annotation = wfdb.rdann(os.path.join(data_path, record), 'atr')
    
    # Use only one channel (e.g., channel 0)
    signal = signal[:, 0].reshape(-1, 1)
    
    # Normalize the signal
    scaler = StandardScaler()
    signal = scaler.fit_transform(signal)
    
    # Segment the signal
    segments = []
    labels = []
    for i in range(len(annotation.sample)):
        if annotation.sample[i] - 99 > 0 and annotation.sample[i] + 160 < len(signal):
            segments.append(signal[annotation.sample[i] - 99 : annotation.sample[i] + 161])
            labels.append(annotation.symbol[i])
    
    return np.array(segments), np.array(labels)

# Function to load and preprocess all records in the dataset
def load_and_preprocess_all_records(data_path):
    all_segments = []
    all_labels = []
    
    for record in os.listdir(data_path):
        if record.endswith('.dat'):
            record_name = record[:-4]  # Remove the file extension
            segments, labels = load_and_preprocess(record_name)
            all_segments.append(segments)
            all_labels.append(labels)
    
    # Concatenate all segments and labels
    all_segments = np.vstack(all_segments)
    all_labels = np.concatenate(all_labels)
    
    return all_segments, all_labels

# Load and preprocess the entire dataset
segments, labels = load_and_preprocess_all_records(data_path)

# Filter out unwanted labels (keeping only certain labels, e.g., 'N', 'A', 'L', 'R', 'V')
valid_labels = ['N', 'A', 'L', 'R', 'V']
mask = np.isin(labels, valid_labels)
segments = segments[mask]
labels = labels[mask]

# Reshape segments to fit the model's expected input shape
segments = segments.reshape(segments.shape[0], segments.shape[1], 1)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to one-hot encoding for MSE models
labels_one_hot = to_categorical(labels_encoded)

# Split the data into training, validation, and test sets (80%, 10%, 10%)
X_train, X_temp, y_train, y_temp = train_test_split(segments, labels_encoded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Separate one-hot encoded labels for MSE models
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)
y_test_one_hot = to_categorical(y_test)

# Functions to create the NN models
def create_nn_model(input_shape, activation, optimizer, loss):
    model = Sequential([
        Input(shape=input_shape),
        Flatten(),
        Dense(128, activation=activation),
        Dense(5, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Functions to create the DNN models
def create_dnn_model(input_shape, layers, activation, optimizer, loss):
    model = Sequential([
        Input(shape=input_shape),
        Flatten()
    ])
    for _ in range(layers):
        model.add(Dense(128, activation=activation))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Create NN models with specified configurations
nn_models = [
    create_nn_model((260, 1), 'sigmoid', 'sgd', 'mean_squared_error'),
    create_nn_model((260, 1), 'relu', 'sgd', 'mean_squared_error'),
    create_nn_model((260, 1), 'relu', 'adam', 'mean_squared_error'),
    create_nn_model((260, 1), 'relu', 'adam', 'sparse_categorical_crossentropy')
]
nn_model_names = ['NN-1', 'NN-2', 'NN-3', 'NN-4']
nn_epochs = [1000, 600, 10, 10]

# Create DNN models with specified configurations
dnn_models = [
    create_dnn_model((260, 1), 2, 'relu', 'adam', 'sparse_categorical_crossentropy'),
    create_dnn_model((260, 1), 3, 'relu', 'adam', 'sparse_categorical_crossentropy'),
    create_dnn_model((260, 1), 5, 'relu', 'adam', 'sparse_categorical_crossentropy')
]
dnn_model_names = ['DNN-1', 'DNN-2', 'DNN-3']
dnn_epochs = [10, 10, 10]

# Train and evaluate each NN model
for model, name, epochs in zip(nn_models, nn_model_names, nn_epochs):
    print(f'Training {name}...')
    start_time = time.time()
    
    # Select appropriate labels for the model
    if name in ['NN-1', 'NN-2', 'NN-3']:
        y_train_labels = y_train_one_hot
        y_val_labels = y_val_one_hot
        y_test_labels = y_test_one_hot
    else:
        y_train_labels = y_train
        y_val_labels = y_val
        y_test_labels = y_test
    
    # Train the model
    history = model.fit(X_train, y_train_labels, epochs=epochs, validation_data=(X_val, y_val_labels), batch_size=128)
    
    # Calculate the total training time
    total_training_time = time.time() - start_time
    
    # Predict on the test data
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_classes)
    
    # Compute the metrics
    overall_accuracy = np.sum(y_pred_classes == y_test) / len(y_test)
    overall_sensitivity = recall_score(y_test, y_pred_classes, average='macro') * 100
    overall_specificity = (conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])) * 100 if conf_matrix.shape[0] > 1 else 0
    overall_precision = precision_score(y_test, y_pred_classes, average='macro') * 100
    overall_fscore = f1_score(y_test, y_pred_classes, average='macro') * 100
    
    # Print the results
    print(f"Results for {name}:")
    print(f"Total Training Time: {total_training_time:.2f} seconds")
    print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")
    print(f"Overall Sensitivity: {overall_sensitivity:.2f}%")
    print(f"Overall Specificity: {overall_specificity:.2f}%")
    print(f"Overall Precision: {overall_precision:.2f}%")
    print(f"Overall F-Score: {overall_fscore:.2f}%")
    
    # Plot training & validation loss values
    plt.figure(figsize=(5, 5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Train and evaluate each DNN model
for model, name, epochs in zip(dnn_models, dnn_model_names, dnn_epochs):
    print(f'Training {name}...')
    start_time = time.time()
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=128)
    
    # Calculate the total training time
    total_training_time = time.time() - start_time
    
    # Predict on the test data
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_classes)
    
    # Compute the metrics
    overall_accuracy = np.sum(y_pred_classes == y_test) / len(y_test)
    overall_sensitivity = recall_score(y_test, y_pred_classes, average='macro') * 100
    overall_specificity = (conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])) * 100 if conf_matrix.shape[0] > 1 else 0
    overall_precision = precision_score(y_test, y_pred_classes, average='macro') * 100
    overall_fscore = f1_score(y_test, y_pred_classes, average='macro') * 100
    
    # Print the results
    print(f"Results for {name}:")
    print(f"Total Training Time: {total_training_time:.2f} seconds")
    print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")
    print(f"Overall Sensitivity: {overall_sensitivity:.2f}%")
    print(f"Overall Specificity: {overall_specificity:.2f}%")
    print(f"Overall Precision: {overall_precision:.2f}%")
    print(f"Overall F-Score: {overall_fscore:.2f}%")
    
    # Plot training & validation loss values
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper left')

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper left')

    plt.show()


### CNNs

In [ ]:
# Define the path to the extracted dataset
data_path = 'M:\Dissertation\mit-bih-arrhythmia-database-1.0.0-20240722T094228Z-001\mit-bih-arrhythmia-database-1.0.0'

# Function to load a record and preprocess
def load_and_preprocess(record):
    signal, fields = wfdb.rdsamp(os.path.join(data_path, record))
    annotation = wfdb.rdann(os.path.join(data_path, record), 'atr')
    
    # Use only one channel (e.g., channel 0)
    signal = signal[:, 0].reshape(-1, 1)
    
    # Normalize the signal
    scaler = StandardScaler()
    signal = scaler.fit_transform(signal)
    
    # Segment the signal
    segments = []
    labels = []
    for i in range(len(annotation.sample)):
        if annotation.sample[i] - 99 > 0 and annotation.sample[i] + 160 < len(signal):
            segments.append(signal[annotation.sample[i] - 99 : annotation.sample[i] + 161])
            labels.append(annotation.symbol[i])
    
    return np.array(segments), np.array(labels)

# Function to load and preprocess all records in the dataset
def load_and_preprocess_all_records(data_path):
    all_segments = []
    all_labels = []
    
    for record in os.listdir(data_path):
        if record.endswith('.dat'):
            record_name = record[:-4]  # Remove the file extension
            segments, labels = load_and_preprocess(record_name)
            all_segments.append(segments)
            all_labels.append(labels)
    
    # Concatenate all segments and labels
    all_segments = np.vstack(all_segments)
    all_labels = np.concatenate(all_labels)
    
    return all_segments, all_labels

# Load and preprocess the entire dataset
segments, labels = load_and_preprocess_all_records(data_path)

# Filter out unwanted labels (keeping only certain labels, e.g., 'N', 'L', 'R', 'A', 'V')
valid_labels = ['N', 'L', 'R', 'A', 'V']
mask = np.isin(labels, valid_labels)
segments = segments[mask]
labels = labels[mask]

# Reshape segments to fit the model's expected input shape
segments = segments.reshape(segments.shape[0], segments.shape[1], 1, 1)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(segments, labels_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Functions to create the CNN models
def create_cnn_1(input_shape):
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        Conv2D(32, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dense(64, activation='relu'),
        Dense(5, activation='softmax')
    ])
    return model

def create_cnn_2(input_shape):
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        Conv2D(32, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(4032, activation='relu'),
        Dense(64, activation='relu'),
        Dense(5, activation='softmax')
    ])
    return model

def create_cnn_3(input_shape):
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        Conv2D(32, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(128, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(3712, activation='relu'),
        Dense(64, activation='relu'),
        Dense(5, activation='softmax')
    ])
    return model

def create_cnn_4(input_shape):
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        Conv2D(32, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(128, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(256, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Flatten(),
        Dense(3328, activation='relu'),
        Dense(64, activation='relu'),
        Dense(5, activation='softmax')
    ])
    return model

input_shape = (260, 1, 1)  # Adjust this based on the actual input shape

# Creating the models
models = [create_cnn_1(input_shape), create_cnn_2(input_shape), create_cnn_3(input_shape), create_cnn_4(input_shape)]
model_names = ['CNN-1', 'CNN-2', 'CNN-3', 'CNN-4']

# Compile the models
for model in models:
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train and evaluate each model
performance_results = []

for model, name in zip(models, model_names):
    print(f'Training {name}...')
    start_time = time.time()
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=256)
    
    # Calculate the total training time
    total_training_time = time.time() - start_time
    
    # Predict on the test data
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_classes)
    
    # Compute the metrics
    overall_accuracy = np.sum(y_pred_classes == y_test) / len(y_test)
    overall_sensitivity = recall_score(y_test, y_pred_classes, average='macro') * 100
    overall_specificity = (conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])) * 100 if conf_matrix.shape[0] > 1 else 0
    overall_precision = precision_score(y_test, y_pred_classes, average='macro') * 100
    overall_fscore = f1_score(y_test, y_pred_classes, average='macro') * 100
    
    # Store the results for comparison
    performance_results.append({
        'Model': name,
        'Accuracy (%)': overall_accuracy * 100,
        'Sensitivity (%)': overall_sensitivity,
        'Specificity (%)': overall_specificity,
        'Precision (%)': overall_precision,
        'F1-Score (%)': overall_fscore,
        'Training Time (s)': total_training_time
    })
    
    # Print the results
    print(f"Results for {name}:")
    print(f"Total Training Time: {total_training_time:.2f} seconds")
    print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")
    print(f"Overall Sensitivity: {overall_sensitivity:.2f}%")
    print(f"Overall Specificity: {overall_specificity:.2f}%")
    print(f"Overall Precision: {overall_precision:.2f}%")
    print(f"Overall F-Score: {overall_fscore:.2f}%")
    
    # Plot training & validation accuracy values (as percentage)
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(np.array(history.history['accuracy']) * 100, 'o-')
    plt.plot(np.array(history.history['val_accuracy']) * 100, 'o-')
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend(['Train', 'Val'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], 'o-')
    plt.plot(history.history['val_loss'], 'o-')
    plt.title(f'{name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper left')

    plt.show()

    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=valid_labels, yticklabels=valid_labels)
    plt.title(f'{name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

# Convert the performance results to a DataFrame for easy comparison
performance_df = pd.DataFrame(performance_results)

# Print the comparison table
print("Performance Comparison Table:")
print(performance_df)

# Additional comparison between CNN-1 and CNN-4 for specific classes
class_indices = [valid_labels.index(label) for label in ['A', 'L', 'N', 'R', 'V']]
comparison_results = []

for name, model in zip(['CNN-1', 'CNN-4'], [models[0], models[3]]):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Calculate metrics for each class
    class_comparison = {
        'Model': name,
        'APB': {
            'Acc': np.sum((y_test == class_indices[0]) & (y_pred_classes == class_indices[0])) / np.sum(y_test == class_indices[0]) * 100,
            'Sen': recall_score(y_test == class_indices[0], y_pred_classes == class_indices[0]) * 100,
            'Spe': (conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])) * 100,
            'Prec': precision_score(y_test == class_indices[0], y_pred_classes == class_indices[0]) * 100,
            'F-Score': f1_score(y_test == class_indices[0], y_pred_classes == class_indices[0]) * 100,
        },
        'LBBB': {
            'Acc': np.sum((y_test == class_indices[1]) & (y_pred_classes == class_indices[1])) / np.sum(y_test == class_indices[1]) * 100,
            'Sen': recall_score(y_test == class_indices[1], y_pred_classes == class_indices[1]) * 100,
            'Spe': (conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[1,0])) * 100,
            'Prec': precision_score(y_test == class_indices[1], y_pred_classes == class_indices[1]) * 100,
            'F-Score': f1_score(y_test == class_indices[1], y_pred_classes == class_indices[1]) * 100,
        },
        'N': {
            'Acc': np.sum((y_test == class_indices[2]) & (y_pred_classes == class_indices[2])) / np.sum(y_test == class_indices[2]) * 100,
            'Sen': recall_score(y_test == class_indices[2], y_pred_classes == class_indices[2]) * 100,
            'Spe': (conf_matrix[2,2] / (conf_matrix[2,2] + conf_matrix[2,1])) * 100,
            'Prec': precision_score(y_test == class_indices[2], y_pred_classes == class_indices[2]) * 100,
            'F-Score': f1_score(y_test == class_indices[2], y_pred_classes == class_indices[2]) * 100,
        },
        'RBBB': {
            'Acc': np.sum((y_test == class_indices[3]) & (y_pred_classes == class_indices[3])) / np.sum(y_test == class_indices[3]) * 100,
            'Sen': recall_score(y_test == class_indices[3], y_pred_classes == class_indices[3]) * 100,
            'Spe': (conf_matrix[3,3] / (conf_matrix[3,3] + conf_matrix[3,0])) * 100,
            'Prec': precision_score(y_test == class_indices[3], y_pred_classes == class_indices[3]) * 100,
            'F-Score': f1_score(y_test == class_indices[3], y_pred_classes == class_indices[3]) * 100,
        },
        'PVC': {
            'Acc': np.sum((y_test == class_indices[4]) & (y_pred_classes == class_indices[4])) / np.sum(y_test == class_indices[4]) * 100,
            'Sen': recall_score(y_test == class_indices[4], y_pred_classes == class_indices[4]) * 100,
            'Spe': (conf_matrix[4,4] / (conf_matrix[4,4] + conf_matrix[4,0])) * 100,
            'Prec': precision_score(y_test == class_indices[4], y_pred_classes == class_indices[4]) * 100,
            'F-Score': f1_score(y_test == class_indices[4], y_pred_classes == class_indices[4]) * 100,
        }
    }
    
    comparison_results.append(class_comparison)

# Convert the comparison results to a DataFrame for easy display
formatted_df = pd.DataFrame()

for i, class_label in enumerate(['APB', 'LBBB', 'N', 'RBBB', 'PVC']):
    for metric in ['Acc', 'Sen', 'Spe', 'Prec', 'F-Score']:
        formatted_df[f'{class_label} {metric} (%)'] = [comparison_results[0][class_label][metric], comparison_results[1][class_label][metric]]

formatted_df.index = ['CNN-1', 'CNN-4']

print("Class Comparison Between CNN-1 and CNN-4:")
print(formatted_df)


### LSTM

In [ ]:
# Define the path to the extracted dataset
data_path = 'M:\Dissertation\mit-bih-arrhythmia-database-1.0.0-20240722T094228Z-001\mit-bih-arrhythmia-database-1.0.0'

# Function to load a record and preprocess
def load_and_preprocess(record):
    signal, fields = wfdb.rdsamp(os.path.join(data_path, record))
    annotation = wfdb.rdann(os.path.join(data_path, record), 'atr')
    
    # Use only one channel (e.g., channel 0)
    signal = signal[:, 0].reshape(-1, 1)
    
    # Segment the signal
    segments = []
    labels = []
    for i in range(len(annotation.sample)):
        if annotation.sample[i] - 99 > 0 and annotation.sample[i] + 160 < len(signal):
            segments.append(signal[annotation.sample[i] - 99 : annotation.sample[i] + 161])
            labels.append(annotation.symbol[i])
    
    return np.array(segments), np.array(labels)

# Function to load and preprocess all records in the dataset
def load_and_preprocess_all_records(data_path):
    all_segments = []
    all_labels = []
    
    for record in os.listdir(data_path):
        if record.endswith('.dat'):
            record_name = record[:-4]  # Remove the file extension
            segments, labels = load_and_preprocess(record_name)
            all_segments.append(segments)
            all_labels.append(labels)
    
    # Concatenate all segments and labels
    all_segments = np.vstack(all_segments)
    all_labels = np.concatenate(all_labels)
    
    return all_segments, all_labels

# Load and preprocess the entire dataset
segments, labels = load_and_preprocess_all_records(data_path)

# Filter out unwanted labels (keeping only certain labels, e.g., 'N', 'L', 'R', 'A', 'V')
valid_labels = ['N', 'L', 'R', 'A', 'V']
mask = np.isin(labels, valid_labels)
segments = segments[mask]
labels = labels[mask]

# Reshape segments to fit the model's expected input shape
segments = segments.reshape(segments.shape[0], segments.shape[1], 1)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to categorical format
labels_categorical = tf.keras.utils.to_categorical(labels_encoded)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(segments, labels_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# 1. Vanilla LSTM Model (return_sequences=False)
def create_vanilla_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(32, return_sequences=False, input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 2. Vanilla LSTM Model (return_sequences=True)
def create_vanilla_lstm_seq(input_shape, num_classes):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 3. Stacked LSTM Model
def create_stacked_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 4. Stacked Bidirectional LSTM Model
def create_stacked_blstm(input_shape, num_classes):
    model = Sequential([
        Bidirectional(LSTM(32, return_sequences=True), input_shape=input_shape),
        Bidirectional(LSTM(32)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the input shape based on your data
input_shape = (segments.shape[1], 1)  # (timesteps, features)
num_classes = len(valid_labels)

# Create the models
vanilla_lstm_model = create_vanilla_lstm(input_shape, num_classes)
vanilla_lstm_seq_model = create_vanilla_lstm_seq(input_shape, num_classes)
stacked_lstm_model = create_stacked_lstm(input_shape, num_classes)
stacked_blstm_model = create_stacked_blstm(input_shape, num_classes)

# Define the models and epochs for training
models = [
    (vanilla_lstm_model, 'Vanilla LSTM', 25),
    (vanilla_lstm_seq_model, 'Vanilla LSTM (return_sequences=True)', 12),
    (stacked_lstm_model, 'Stacked LSTM', 12),
    (stacked_blstm_model, 'Bidirectional LSTM', 10)
]

# Train the models and evaluate
for model, name, epochs in models:
    print(f'Training {name}...')
    start_time = time.time()
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=32)
    
    # Calculate the total training time
    total_training_time = time.time() - start_time
    
    # Predict on the test data
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Compute the metrics
    overall_accuracy = np.sum(y_pred_classes == y_test_classes) / len(y_test_classes)
    overall_sensitivity = recall_score(y_test_classes, y_pred_classes, average='macro') * 100
    overall_specificity = (conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])) * 100 if conf_matrix.shape[0] > 1 else 0
    overall_precision = precision_score(y_test_classes, y_pred_classes, average='macro') * 100
    overall_fscore = f1_score(y_test_classes, y_pred_classes, average='macro') * 100
    
    # Print the results
    print(f"Results for {name}:")
    print(f"Total Training Time: {total_training_time:.2f} seconds")
    print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")
    print(f"Overall Sensitivity: {overall_sensitivity:.2f}%")
    print(f"Overall Specificity: {overall_specificity:.2f}%")
    print(f"Overall Precision: {overall_precision:.2f}%")
    print(f"Overall F-Score: {overall_fscore:.2f}%")
    
    # Plot training & validation accuracy values (as percentage)
    plt.figure(figsize=(7, 5))
    plt.plot(np.array(history.history['accuracy']) * 100, 'o-')
    plt.plot(np.array(history.history['val_accuracy']) * 100, 'o-')
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
    plt.grid(True)
    plt.show()


### CNN-LSTM

In [ ]:
# Define the path to the extracted dataset
data_path = 'M:\Dissertation\mit-bih-arrhythmia-database-1.0.0-20240722T094228Z-001\mit-bih-arrhythmia-database-1.0.0'

# Function to load a record and preprocess
def load_and_preprocess(record):
    signal, fields = wfdb.rdsamp(os.path.join(data_path, record))
    annotation = wfdb.rdann(os.path.join(data_path, record), 'atr')
    
    # Use only one channel (e.g., channel 0)
    signal = signal[:, 0].reshape(-1, 1)
    
    # Segment the signal
    segments = []
    labels = []
    for i in range(len(annotation.sample)):
        if annotation.sample[i] - 99 > 0 and annotation.sample[i] + 160 < len(signal):
            segments.append(signal[annotation.sample[i] - 99 : annotation.sample[i] + 161])
            labels.append(annotation.symbol[i])
    
    return np.array(segments), np.array(labels)

# Function to load and preprocess all records in the dataset
def load_and_preprocess_all_records(data_path):
    all_segments = []
    all_labels = []
    
    for record in os.listdir(data_path):
        if record.endswith('.dat'):
            record_name = record[:-4]  # Remove the file extension
            segments, labels = load_and_preprocess(record_name)
            all_segments.append(segments)
            all_labels.append(labels)
    
    # Concatenate all segments and labels
    all_segments = np.vstack(all_segments)
    all_labels = np.concatenate(all_labels)
    
    return all_segments, all_labels

# Load and preprocess the entire dataset
segments, labels = load_and_preprocess_all_records(data_path)

# Filter out unwanted labels (keeping only certain labels, e.g., 'N', 'L', 'R', 'A', 'V')
valid_labels = ['N', 'L', 'R', 'A', 'V']
mask = np.isin(labels, valid_labels)
segments = segments[mask]
labels = labels[mask]

# Reshape segments to fit the model's expected input shape
segments = segments.reshape(segments.shape[0], segments.shape[1], 1, 1)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to categorical format
labels_categorical = tf.keras.utils.to_categorical(labels_encoded)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(segments, labels_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 1. Vanilla LSTM Model
def create_vanilla_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(32, input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 2. Stacked LSTM Model
def create_stacked_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 3. Stacked Bidirectional LSTM Model
def create_stacked_blstm(input_shape, num_classes):
    model = Sequential([
        Bidirectional(LSTM(32, return_sequences=True), input_shape=input_shape),
        Bidirectional(LSTM(32)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 4. CNN-LSTM Model
def create_cnn_lstm(input_shape, num_classes):
    model = Sequential([
        # CNN part
        Conv2D(32, (5, 1), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(128, (5, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        Conv2D(256, (3, 1), activation='relu'),
        MaxPooling2D((2, 1)),
        
        # Reshape for LSTM input
        Reshape((13, 256)),  # Adjusting the shape for LSTM input
        
        # LSTM part
        LSTM(32, return_sequences=False),
        
        # Dense layers
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the input shape based on your data
input_shape_lstm = (segments.shape[1], 1)  # (timesteps, features) for LSTM
input_shape_cnn = (segments.shape[1], 1, 1)  # (timesteps, features, 1) for CNN

# Number of classes
num_classes = len(valid_labels)

# Create the models
vanilla_lstm_model = create_vanilla_lstm(input_shape_lstm, num_classes)
stacked_lstm_model = create_stacked_lstm(input_shape_lstm, num_classes)
stacked_blstm_model = create_stacked_blstm(input_shape_lstm, num_classes)
cnn_lstm_model = create_cnn_lstm(input_shape_cnn, num_classes)

# Train the models (using pre-trained models if available)
# For demonstration, I'll just run the training loop again for each model.

models = [
    (vanilla_lstm_model, 'Vanilla LSTM', 25),
    (stacked_lstm_model, 'Stacked LSTM', 12),
    (stacked_blstm_model, 'Bidirectional LSTM', 10),
    (cnn_lstm_model, 'CNN-LSTM', 14)
]

for model, name, epochs in models:
    print(f'Training {name}...')
    start_time = time.time()
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=32)
    total_training_time = time.time() - start_time
    print(f"Total Training Time for {name}: {total_training_time:.2f} seconds")

# Predict using all models
y_pred_vanilla = vanilla_lstm_model.predict(X_test)
y_pred_stacked = stacked_lstm_model.predict(X_test)
y_pred_bidirectional = stacked_blstm_model.predict(X_test)
y_pred_cnn_lstm = cnn_lstm_model.predict(X_test)

# Ensemble method - Average predictions
y_pred_ensemble = (y_pred_vanilla + y_pred_stacked + y_pred_bidirectional + y_pred_cnn_lstm) / 4
y_pred_classes_ensemble = np.argmax(y_pred_ensemble, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Compute the confusion matrix
conf_matrix_ensemble = confusion_matrix(y_test_classes, y_pred_classes_ensemble)

# Compute the metrics
overall_accuracy_ensemble = np.sum(y_pred_classes_ensemble == y_test_classes) / len(y_test_classes)
overall_sensitivity_ensemble = recall_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100
overall_specificity_ensemble = (conf_matrix_ensemble[0,0] / (conf_matrix_ensemble[0,0] + conf_matrix_ensemble[0,1])) * 100 if conf_matrix_ensemble.shape[0] > 1 else 0
overall_precision_ensemble = precision_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100
overall_fscore_ensemble = f1_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100

# Print the ensemble results
print(f"Results for Ensemble of All Models:")
print(f"Overall Accuracy: {overall_accuracy_ensemble * 100:.2f}%")
print(f"Overall Sensitivity: {overall_sensitivity_ensemble:.2f}%")
print(f"Overall Specificity: {overall_specificity_ensemble:.2f}%")
print(f"Overall Precision: {overall_precision_ensemble:.2f}%")
print(f"Overall F-Score: {overall_fscore_ensemble:.2f}%")

# Calculate per-class metrics
class_names = ['APB', 'LBBB', 'NSR', 'RBBB', 'PVC']
accuracy_per_class_ensemble = []
sensitivity_per_class_ensemble = []
specificity_per_class_ensemble = []
precision_per_class_ensemble = []
f1score_per_class_ensemble = []

for i, class_name in enumerate(class_names):
    true_positive = conf_matrix_ensemble[i, i]
    false_positive = np.sum(conf_matrix_ensemble[:, i]) - true_positive
    false_negative = np.sum(conf_matrix_ensemble[i, :]) - true_positive
    true_negative = np.sum(conf_matrix_ensemble) - (true_positive + false_positive + false_negative)
    
    accuracy = (true_positive + true_negative) / np.sum(conf_matrix_ensemble) * 100
    sensitivity = true_positive / (true_positive + false_negative) * 100 if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) * 100 if (true_negative + false_positive) > 0 else 0
    precision = true_positive / (true_positive + false_positive) * 100 if (true_positive + false_positive) > 0 else 0
    f1score = (2 * precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
    
    accuracy_per_class_ensemble.append(accuracy)
    sensitivity_per_class_ensemble.append(sensitivity)
    specificity_per_class_ensemble.append(specificity)
    precision_per_class_ensemble.append(precision)
    f1score_per_class_ensemble.append(f1score)

# Display the ensemble results in a DataFrame
performance_df_ensemble = pd.DataFrame({
    'Classes': class_names,
    'Accuracy (%)': accuracy_per_class_ensemble,
    'Sensitivity (%)': sensitivity_per_class_ensemble,
    'Specificity (%)': specificity_per_class_ensemble,
    'Precision (%)': precision_per_class_ensemble,
    'F1 Score (%)': f1score_per_class_ensemble
})

# Calculate overall accuracy to match the table
performance_df_ensemble['Overall Accuracy (%)'] = overall_accuracy_ensemble * 100

print(performance_df_ensemble)

# Plot ensemble confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_ensemble, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'Ensemble Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### Stacked GRU with CNN-LSTM

In [ ]:
import os
import wfdb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GRU, Reshape, Input, Flatten, LSTM, Dropout
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Define the path to the extracted dataset
data_path = 'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0'

# Function to load a record and preprocess
def load_and_preprocess(record):
    signal, fields = wfdb.rdsamp(os.path.join(data_path, record))
    annotation = wfdb.rdann(os.path.join(data_path, record), 'atr')
   
    # Use only one channel (e.g., channel 0)
    signal = signal[:, 0].reshape(-1, 1)
   
    # Segment the signal
    segments = []
    labels = []
    for i in range(len(annotation.sample)):
        if annotation.sample[i] - 99 > 0 and annotation.sample[i] + 160 < len(signal):
            segments.append(signal[annotation.sample[i] - 99 : annotation.sample[i] + 161])
            labels.append(annotation.symbol[i])
   
    return np.array(segments), np.array(labels)

# Function to load and preprocess all records in the dataset
def load_and_preprocess_all_records(data_path):
    all_segments = []
    all_labels = []
   
    for record in os.listdir(data_path):
        if record.endswith('.dat'):
            record_name = record[:-4]  # Remove the file extension
            segments, labels = load_and_preprocess(record_name)
            all_segments.append(segments)
            all_labels.append(labels)
   
    # Concatenate all segments and labels
    all_segments = np.vstack(all_segments)
    all_labels = np.concatenate(all_labels)
   
    return all_segments, all_labels

# Load and preprocess the entire dataset
segments, labels = load_and_preprocess_all_records(data_path)

# Filter out unwanted labels (keeping only certain labels, e.g., 'N', 'L', 'R', 'A', 'V')
valid_labels = ['N', 'L', 'R', 'A', 'V']
mask = np.isin(labels, valid_labels)
segments = segments[mask]
labels = labels[mask]

# Reshape segments to fit the model's expected input shape
segments = segments.reshape(segments.shape[0], segments.shape[1], 1, 1)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to categorical format
labels_categorical = tf.keras.utils.to_categorical(labels_encoded)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(segments, labels_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 1. Stacked GRU Model with Regularization
def create_stacked_gru(input_shape, num_classes):
    model = Sequential([
        GRU(32, return_sequences=True, input_shape=input_shape),
        GRU(32),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 2. CNN-LSTM Model with Regularization
def create_cnn_lstm(input_shape_cnn, input_shape_lstm, num_classes):
    cnn_model = Sequential([
        Conv2D(32, (5, 1), activation='relu', input_shape=input_shape_cnn),
        MaxPooling2D((2, 1)),
        Conv2D(64, (3, 1), activation='relu'),
        Flatten(),
        Dense(260 * 32, activation='relu')  # Adjusted to match the LSTM input
    ])
   
    cnn_input = Input(shape=input_shape_cnn)
    cnn_output = cnn_model(cnn_input)
   
    # Correctly reshape the CNN output to match LSTM input expectations
    timesteps = input_shape_lstm[0]  # This is typically the length of the input sequence
    features = 32  # Reduced the features to match the Dense layer output
   
    lstm_input = Reshape((timesteps, features))(cnn_output)
    lstm_output = LSTM(32)(lstm_input)
   
    output = Dense(num_classes, activation='softmax')(lstm_output)
   
    model = Model(inputs=cnn_input, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the input shape based on your data
input_shape_lstm_gru = (segments.shape[1], 1)  # (timesteps, features) for LSTM/GRU
input_shape_cnn_resnet = (segments.shape[1], 1, 1)  # (timesteps, features, 1) for CNN/ResNet

# Number of classes
num_classes = len(valid_labels)

# Create the models
stacked_gru_model = create_stacked_gru(input_shape_lstm_gru, num_classes)
cnn_lstm_model = create_cnn_lstm(input_shape_cnn_resnet, input_shape_lstm_gru, num_classes)

# Train the models and store history
models = [
    (stacked_gru_model, 'Stacked GRU', 40),
    (cnn_lstm_model, 'CNN-LSTM', 40)
]

histories = {}

for model, name, epochs in models:
    print(f'Training {name}...')
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=32)
    total_training_time = time.time() - start_time
    print(f"Total Training Time for {name}: {total_training_time:.2f} seconds")
    histories[name] = history

# Predict using the models
y_pred_stacked_gru = stacked_gru_model.predict(X_test)
y_pred_cnn_lstm = cnn_lstm_model.predict(X_test)

# Ensemble method - Average predictions from the models
y_pred_ensemble = (y_pred_stacked_gru + y_pred_cnn_lstm) / 2
y_pred_classes_ensemble = np.argmax(y_pred_ensemble, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Compute the confusion matrix
conf_matrix_ensemble = confusion_matrix(y_test_classes, y_pred_classes_ensemble)

# Compute the metrics
overall_accuracy_ensemble = np.sum(y_pred_classes_ensemble == y_test_classes) / len(y_test_classes)
overall_sensitivity_ensemble = recall_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100
overall_specificity_ensemble = (conf_matrix_ensemble[0,0] / (conf_matrix_ensemble[0,0] + conf_matrix_ensemble[0,1])) * 100 if conf_matrix_ensemble.shape[0] > 1 else 0
overall_precision_ensemble = precision_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100
overall_fscore_ensemble = f1_score(y_test_classes, y_pred_classes_ensemble, average='macro') * 100

# Print the ensemble results
print(f"Results for Ensemble of Stacked GRU and CNN-LSTM Models:")
print(f"Overall Accuracy: {overall_accuracy_ensemble * 100:.2f}%")
print(f"Overall Sensitivity: {overall_sensitivity_ensemble:.2f}%")
print(f"Overall Specificity: {overall_specificity_ensemble:.2f}%")
print(f"Overall Precision: {overall_precision_ensemble:.2f}%")
print(f"Overall F-Score: {overall_fscore_ensemble:.2f}%")

# Calculate per-class metrics
class_names = label_encoder.inverse_transform(np.arange(num_classes))
accuracy_per_class_ensemble = []
sensitivity_per_class_ensemble = []
specificity_per_class_ensemble = []
precision_per_class_ensemble = []
f1score_per_class_ensemble = []

for i, class_name in enumerate(class_names):
    true_positive = conf_matrix_ensemble[i, i]
    false_positive = np.sum(conf_matrix_ensemble[:, i]) - true_positive
    false_negative = np.sum(conf_matrix_ensemble[i, :]) - true_positive
    true_negative = np.sum(conf_matrix_ensemble) - (true_positive + false_positive + false_negative)
   
    accuracy = (true_positive + true_negative) / np.sum(conf_matrix_ensemble) * 100
    sensitivity = true_positive / (true_positive + false_negative) * 100 if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) * 100 if (true_negative + false_positive) > 0 else 0
    precision = true_positive / (true_positive + false_positive) * 100 if (true_positive + false_positive) > 0 else 0
    f1score = (2 * precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
   
    accuracy_per_class_ensemble.append(accuracy)
    sensitivity_per_class_ensemble.append(sensitivity)
    specificity_per_class_ensemble.append(specificity)
    precision_per_class_ensemble.append(precision)
    f1score_per_class_ensemble.append(f1score)

# Display the ensemble results in a DataFrame
performance_df_ensemble = pd.DataFrame({
    'Classes': class_names,
    'Accuracy (%)': accuracy_per_class_ensemble,
    'Sensitivity (%)': sensitivity_per_class_ensemble,
    'Specificity (%)': specificity_per_class_ensemble,
    'Precision (%)': precision_per_class_ensemble,
    'F1 Score (%)': f1score_per_class_ensemble
})

# Add overall accuracy to match the table
performance_df_ensemble['Overall Accuracy (%)'] = overall_accuracy_ensemble * 100

print(performance_df_ensemble)

# Plot ensemble confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_ensemble, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'Ensemble Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Plot accuracy and loss graphs for each model
for name, history in histories.items():
    plt.figure(figsize=(10, 5))
   
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(np.array(history.history['accuracy']) * 100, 'o--', label='Train Accuracy')
    plt.plot(np.array(history.history['val_accuracy']) * 100, 'o--', label='Validation Accuracy')
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
   
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], 'o--', label='Train Loss')
    plt.plot(history.history['val_loss'], 'o--', label='Validation Loss')
    plt.title(f'{name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
   
    plt.show()